<a href="https://colab.research.google.com/github/mtareqsoliman/yap506_MatYon/blob/main/YapiSistHesabMatYon_MatrixSoftware_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Due to settlement only

In [12]:
import pandas as pd
import numpy as np
import scipy.special as ssp
import sympy as sym

# Enter no. of element
n_el = 5

# Enter no. of nodes
n_nd = 6



K_global = np.zeros((n_nd*3,n_nd*3)) # Global stiffness matrix is created

nodes_list = pd.DataFrame(data=None, columns=['x_coord','z_coord'], index=np.arange(1,n_nd+1,1))
rstrd_list = pd.DataFrame(data=None, columns=['Rstrd_theta','Rstrd_x','Rstrd_z'], index=np.arange(1,n_nd+1,1))
spring_list = pd.DataFrame(data=None, columns=['Yay_theta','Yay_x','Yay_z'], index=np.arange(1,n_el+1,1))
pntfr_list = pd.DataFrame(data=np.zeros((n_nd,3),dtype='int'), columns=['P_theta','P_x','P_z'], index=np.arange(1,n_nd+1,1))
dstfr_list = pd.DataFrame(data=np.zeros((n_el,2),dtype='int'), columns=['W_x','W_z'], index=np.arange(1,n_el+1,1))
ele_list = pd.DataFrame(data=None, columns=['node_i','node_j','Length','beta','EI','EF'], index=np.arange(1,n_el+1,1))
tmprt_list = pd.DataFrame(data=None, columns=['element','t_up','t_bot','duzgun_t','EI','EF'], index=np.arange(1,n_el+1,1))
deform_list = pd.DataFrame(data=np.zeros((n_nd,3),dtype='int'), columns=['Δ_rot','Δ_x','Δ_z'], index=np.arange(1,n_nd+1,1))


# Enter Stiffness properties
EI_global = 100000
EF_global = 10000000

# Enter nodes coordinates
nodes_list.loc[1] = [4,8]
nodes_list.loc[2] = [10,8]
nodes_list.loc[3] = [0,4]
nodes_list.loc[4] = [4,4]
nodes_list.loc[5] = [4,0]
nodes_list.loc[6] = [16,0]

# Enter elements   i-j-L-β-   EI-       EF
ele_list.loc[1] = [1,2,6,0,EI_global,EF_global]
ele_list.loc[2] = [3,4,4,0,EI_global,EF_global]
ele_list.loc[3] = [5,4,4,270,EI_global,EF_global]
ele_list.loc[4] = [4,1,4,270,EI_global,EF_global]
ele_list.loc[5] = [2,6,10,np.degrees(np.arctan(8/6)),EI_global,EF_global]


# Enter Applied point loads
pntfr_list.loc[1] = [0,0,0]
pntfr_list.loc[2] = [0,0,0]
pntfr_list.loc[3] = [0,0,0]
pntfr_list.loc[4] = [0,0,0]
pntfr_list.loc[5] = [0,0,0]
pntfr_list.loc[6] = [0,0,0]


# Enter boundary conditions (restrained)
rstrd_list.loc[1] = [0,0,0]
rstrd_list.loc[2] = [0,0,0]
rstrd_list.loc[3] = [0,0,1]
rstrd_list.loc[4] = [0,0,0]
rstrd_list.loc[5] = [1,1,1]
rstrd_list.loc[6] = [0,1,1]

rstrd = np.array(rstrd_list).flatten() # try ravel also


# Enter elastic hinge locations (springs)
spring_list.loc[1] = np.array([0.2*EI_global,10**50,10**50])
spring_list.loc[2] = np.array([10**50,10**50,10**50])
spring_list.loc[3] = np.array([10**50,10**50,10**50])
spring_list.loc[4] = np.array([10**50,10**50,10**50])
spring_list.loc[5] = np.array([10**50,10**50,10**50])
spring_list.loc[6] = np.array([10**50,10**50,10**50])


# Enter Applied distributed loads
dstfr_list.loc[1] = [0,0]
dstfr_list.loc[2] = [0,0]
dstfr_list.loc[3] = [0,0]
dstfr_list.loc[4] = [0,0]
dstfr_list.loc[5] = [0,0]

# Enter sıcaklık değişmesi parameters
#tmprt_list.loc[1] =
deform_list.loc[1] = [0,0,0]
deform_list.loc[2] = [0,0,0]
deform_list.loc[3] = [0,0,0]
deform_list.loc[4] = [0,0,0]
deform_list.loc[5] = [0,-0.003,0.004]
deform_list.loc[6] = [0,0,0]
deform = np.array(deform_list).flatten()

p0_global = np.zeros((n_nd*3,1))

q_global = np.array(pntfr_list).reshape(n_nd*3,1)

#for ele in np.arange(1,n_el+1,1):
for ele in np.arange(1,n_el+1,1):
  i = ele_list.loc[ele]['node_i']
  j = ele_list.loc[ele]['node_j']
  EI = ele_list.loc[ele]['EI']
  EF = ele_list.loc[ele]['EF']
  beta = ele_list.loc[ele]['beta']
  L = ele_list.loc[ele]['Length']
  L_x = L*ssp.cosdg(beta)
  L_z = L*ssp.sindg(beta)

  # distributed load
  W_x = dstfr_list.loc[ele]['W_x']
  W_z = dstfr_list.loc[ele]['W_z']

  ## point load
  Pl_M_i,Pl_M_j = pntfr_list.loc[i]['P_theta'],pntfr_list.loc[j]['P_theta']
  Pl_x_i,Pl_x_j = pntfr_list.loc[i]['P_x'],pntfr_list.loc[j]['P_x']
  Pl_z_i,Pl_z_j = pntfr_list.loc[i]['P_z'],pntfr_list.loc[j]['P_z']


  ## deformation due to settlement
  dfrm_theta_i,dfrm_x_i,dfrm_z_i = deform_list.loc[i]['Δ_rot'],deform_list.loc[i]['Δ_x'],deform_list.loc[i]['Δ_z']
  dfrm_theta_j,dfrm_x_j,dfrm_z_j = deform_list.loc[j]['Δ_rot'],deform_list.loc[j]['Δ_x'],deform_list.loc[j]['Δ_z']


  K_ii = np.matrix(([[4*EI/L,0,6*EI/L**2],
                       [0,EF/L,0],
                       [6*EI/L**2,0,12*EI/L**3]]))

  K_jj = np.matrix(([[4*EI/L,0,6*EI/L**2],
                       [0,EF/L,0],
                       [6*EI/L**2,0,12*EI/L**3]]))
  print('K_{}{}_ii ='.format(i,j))
  display(sym.Matrix(K_ii))

  K_ij = np.matrix(([[2*EI/L,0,6*EI/L**2],
                        [0,EF/L,0],
                        [6*EI/L**2,0,12*EI/L**3]]))

  print('K_{}{}_ij ='.format(i,j))
  display(sym.Matrix(K_ij))


  K1_ii = np.float64(K_ii+np.diag(spring_list.loc[i]))
  K1_jj = np.float64(K_jj+np.diag(spring_list.loc[j]))
  K1_ij = np.float64(K_ij)
  K1_ji = K1_ij

  K0 = np.vstack((np.hstack((K_ii,K_ij)),np.hstack((K_ij,K_jj))))
  K1 = np.vstack((np.hstack((K1_ii,K1_ij)),np.hstack((K1_ji,K1_jj))))
  K_lokal = K0-np.dot(np.dot(K0,np.linalg.inv(K1)),K0)
  print('K_{}{}_LOKAL ='.format(i,j))
  display(sym.Matrix(K_lokal))
  K_ii, K_ij, K_ji, K_jj =  K_lokal[0:3,0:3],K_lokal[0:3,3:6],K_lokal[3:6,0:3],K_lokal[3:6,3:6]

  #K_1 = np.diag(np.full((6, 6), 10**50))
  #K_1[np.]

  T2_i = np.matrix(([[1,0,0],
                       [0,-ssp.cosdg(beta),ssp.sindg(beta)],
                       [0,-ssp.sindg(beta),-ssp.cosdg(beta)]]))

  T2_j = np.matrix(([[1,0,0],
                       [0,ssp.cosdg(beta),-ssp.sindg(beta)],
                       [0,ssp.sindg(beta),ssp.cosdg(beta)]]))

  print('T_2_{} ='.format(ele))
  T_2 =  np.vstack((np.hstack((T2_i,np.zeros((3,3)))),np.hstack((np.zeros((3,3)),T2_j))))
  display(sym.Matrix(T_2))

  # Calculating FEM from distributed load
  p0_dst = np.transpose(np.matrix([-W_x*L_z**2/12+W_z*L_x**2/12,
                               W_x*L_x/2+W_z*L_z/2,
                               W_x*L_z/2+W_z*L_x/2,
                               W_x*L_x**2/12-W_z*L_x**2/12+Pl_M_j,
                              -W_x*L_x/2-W_z*L_z/2,
                              -W_x*L_z/2-W_z*L_x/2]))
  p0_indrgnms = p0_dst-np.dot(np.dot(K0,np.linalg.inv(K1)),p0_dst)

  # calculating P from deformation
  w_dfrm = np.transpose(np.hstack((deform_list.loc[i],deform_list.loc[j])))
  print('w_dfrm_{} ='.format(ele))
  print(w_dfrm)

  p0_dfrm = np.dot(np.dot(np.dot(T_2,K_lokal),np.transpose(T_2)),np.transpose(w_dfrm))
  p0 = p0_indrgnms+p0_dfrm
  p0_transf = p0

  p0_global[(i-1)*3] =     p0_transf[0,0]+p0_global[(i-1)*3]
  p0_global[((i-1)*3)+1] = p0_transf[0,1]+p0_global[((i-1)*3)+1]
  p0_global[((i-1)*3)+2] = p0_transf[0,2]+p0_global[((i-1)*3)+2]
  p0_global[(j-1)*3] =     p0_transf[0,3]+p0_global[(j-1)*3]
  p0_global[((j-1)*3)+1] = p0_transf[0,4]+p0_global[((j-1)*3)+1]
  p0_global[((j-1)*3)+2] = p0_transf[0,5]+p0_global[((j-1)*3)+2]

  print('―' * 50)


  K_global[3*(i-1):3*(i-1)+3,3*(i-1):3*(i-1)+3] = np.dot(np.dot(T2_i,K_ii),np.transpose(T2_i))+K_global[3*(i-1):3*(i-1)+3,3*(i-1):3*(i-1)+3]
  K_global[3*(i-1):3*(i-1)+3,3*(j-1):3*(j-1)+3] = np.dot(np.dot(T2_i,K_ij),np.transpose(T2_j))+K_global[3*(i-1):3*(i-1)+3,3*(j-1):3*(j-1)+3]
  K_global[3*(j-1):3*(j-1)+3,3*(i-1):3*(i-1)+3] = np.dot(np.dot(T2_j,K_ji),np.transpose(T2_i))+K_global[3*(j-1):3*(j-1)+3,3*(i-1):3*(i-1)+3]
  K_global[3*(j-1):3*(j-1)+3,3*(j-1):3*(j-1)+3] = np.dot(np.dot(T2_j,K_jj),np.transpose(T2_j))+K_global[3*(j-1):3*(j-1)+3,3*(j-1):3*(j-1)+3]

  display(sym.Matrix(K_global))
  display(sym.Matrix(p0_global))


  ### force vector
  #p0[i-1:i+2] = [-dstfr_list.loc[ele]['W_x']+dstfr_list.loc[ele]['W_z']+pntfr_list.loc[ele]['P_theta']]


## trimming the equation
trim_index = np.flatnonzero(rstrd)
K_global_trimmed = np.delete(np.delete(K_global,trim_index,0),trim_index,1)

p0_global_trimmed = np.delete(p0_global,trim_index,0)
q_global_trimmed = np.delete(q_global,trim_index,0)

d = np.linalg.solve(K_global_trimmed,q_global_trimmed-p0_global_trimmed)
display(sym.Matrix(d))
print(d)

print('K_global_trimmed')


K_12_ii =


Matrix([
[66666.6666666667,                0, 16666.6666666667],
[               0, 1666666.66666667,                0],
[16666.6666666667,                0, 5555.55555555556]])

K_12_ij =


Matrix([
[33333.3333333333,                0, 16666.6666666667],
[               0, 1666666.66666667,                0],
[16666.6666666667,                0, 5555.55555555556]])

K_12_LOKAL =


...

w_dfrm_5 =
[0. 0. 0. 0. 0. 0.]
――――――――――――――――――――――――――――――――――――――――――――――――――


Matrix([
[  32051.282051282,            6250.0, -3846.15384615385,  7692.30769230769,                 0,  3846.15384615385,        0,          0,        0, 8333.33333333333,    -6250.0,          0,        0,        0,          0,       0,         0,         0],
[           6250.0,  1673697.91666667,                 0,                 0, -1666666.66666667,                 0,        0,          0,        0,          21875.0,   -7031.25,          0,        0,        0,          0,       0,         0,         0],
[-3846.15384615385,                 0,  2502350.42735043, -10256.4102564103,                 0, -2350.42735042735,        0,          0,        0,                0,          0, -2500000.0,        0,        0,          0,       0,         0,         0],
[ 7692.30769230769,                 0, -10256.4102564103,  93846.1538461538,            4800.0,  6656.41025641026,        0,          0,        0,                0,          0,          0,        0,        0,          0, 20000.0,   

Matrix([
[       0],
[       0],
[       0],
[       0],
[       0],
[       0],
[       0],
[       0],
[       0],
[   112.5],
[   56.25],
[-10000.0],
[   112.5],
[  -56.25],
[ 10000.0],
[       0],
[       0],
[       0]])

Matrix([
[  0.00021652088344577],
[ -0.00107631653215281],
[  0.00399453661305903],
[ 0.000417625169267964],
[ -0.00107801523334873],
[ 0.000812996672301632],
[ -0.00123712969399947],
[ -0.00210629239579186],
[-0.000522351633033881],
[ -0.00210629239579186],
[   0.0039954813613771],
[ -6.28105612498691e-6]])

[[ 2.16520883e-04]
 [-1.07631653e-03]
 [ 3.99453661e-03]
 [ 4.17625169e-04]
 [-1.07801523e-03]
 [ 8.12996672e-04]
 [-1.23712969e-03]
 [-2.10629240e-03]
 [-5.22351633e-04]
 [-2.10629240e-03]
 [ 3.99548136e-03]
 [-6.28105612e-06]]
K_global_trimmed


**NOTE: The code below is the same code above, however only inputs are changed**

Due to change in temperature only:

In [13]:
import pandas as pd
import numpy as np
import scipy.special as ssp
import sympy as sym

# Enter no. of element
n_el = 5 # @param {type:"slider", min:1, max:7, step:1}

# Enter no. of nodes
n_nd = 6 # @param {type:"slider", min:2, max:8, step:1}



K_global = np.zeros((n_nd*3,n_nd*3)) # Global stiffness matrix is created

# Enter Stiffness properties
EI_global = 100000 # @param {type:"number"}
EF_global = 10000000 # @param {type:"number"}

nodes_list = pd.DataFrame(data=None, columns=['x_coord','z_coord'], index=np.arange(1,n_nd+1,1))
rstrd_list = pd.DataFrame(data=None, columns=['Rstrd_theta','Rstrd_x','Rstrd_z'], index=np.arange(1,n_nd+1,1))
spring_list = pd.DataFrame(data=None, columns=['Yay_theta','Yay_x','Yay_z'], index=np.arange(1,n_el+1,1))
pntfr_list = pd.DataFrame(data=np.zeros((n_nd,3),dtype='int'), columns=['P_theta','P_x','P_z'], index=np.arange(1,n_nd+1,1))
dstfr_list = pd.DataFrame(data=np.zeros((n_el,2),dtype='int'), columns=['W_x','W_z'], index=np.arange(1,n_el+1,1))
ele_list = pd.DataFrame(data=None, columns=['node_i','node_j','Length','beta','EI','EF'], index=np.arange(1,n_el+1,1))
tmprt_list = pd.DataFrame(data=None, columns=['element','t_up','t_bot','duzgun_t','EI','EF'], index=np.arange(1,n_el+1,1))
deform_list = pd.DataFrame(data=np.zeros((n_nd,3),dtype='int'), columns=['Δ_rot','Δ_x','Δ_z'], index=np.arange(1,n_nd+1,1))

# Enter nodes coordinates
nodes_list.loc[1] = [4,8] # @param {type:"raw"}
nodes_list.loc[2] = [10,8] # @param {type:"raw"}
nodes_list.loc[3] = [0,4] # @param {type:"raw"}
nodes_list.loc[4] = [4,4] # @param {type:"raw"}
nodes_list.loc[5] = [4,0] # @param {type:"raw"}
nodes_list.loc[6] = [16,0] # @param {type:"raw"}

# Enter elements   i-j-L-β-   EI-       EF
ele_list.loc[1] = [1,2,6,0,EI_global,EF_global]  # @param {type:"raw"}
ele_list.loc[2] = [3,4,4,0,EI_global,EF_global]  # @param {type:"raw"}
ele_list.loc[3] = [5,4,4,270,EI_global,EF_global] # @param {type:"raw"}
ele_list.loc[4] = [4,1,4,270,EI_global,EF_global] # @param {type:"raw"}
ele_list.loc[5] = [2,6,10,np.degrees(np.arctan(8/6)),EI_global,EF_global] # @param {type:"raw"}


# Enter Applied point loads
pntfr_list.loc[1] = [0,0,0]  # @param {type:"raw"}
pntfr_list.loc[2] = [0,0,0]  # @param {type:"raw"}
pntfr_list.loc[3] = [0,0,0] # @param {type:"raw"}
pntfr_list.loc[4] = [0,0,0]  # @param {type:"raw"}
pntfr_list.loc[5] = [0,0,0]  # @param {type:"raw"}
pntfr_list.loc[6] = [0,0,0]  # @param {type:"raw"}


# Enter boundary conditions (restrained)
rstrd_list.loc[1] = [0,0,0]  # @param {type:"raw"}
rstrd_list.loc[2] = [0,0,0]  # @param {type:"raw"}
rstrd_list.loc[3] = [0,0,1]  # @param {type:"raw"}
rstrd_list.loc[4] = [0,0,0]  # @param {type:"raw"}
rstrd_list.loc[5] = [1,1,1]  # @param {type:"raw"}
rstrd_list.loc[6] = [0,1,1]  # @param {type:"raw"}

rstrd = np.array(rstrd_list).flatten() # try ravel also


# Enter elastic hinge locations (springs)
spring_list.loc[1] = np.array([0.2*EI_global,10**50,10**50]) # @param {type:"raw"}
spring_list.loc[2] = np.array([10**50,10**50,10**50])      # @param {type:"raw"}
spring_list.loc[3] = np.array([10**50,10**50,10**50])      # @param {type:"raw"}
spring_list.loc[4] = np.array([10**50,10**50,10**50])      # @param {type:"raw"}
spring_list.loc[5] = np.array([10**50,10**50,10**50])      # @param {type:"raw"}
spring_list.loc[6] = np.array([10**50,10**50,10**50])      # @param {type:"raw"}


# Enter Applied distributed loads
dstfr_list.loc[1] = [0,0]       # @param {type:"raw"}
dstfr_list.loc[2] = [0,0]     # @param {type:"raw"}
dstfr_list.loc[3] = [0,0]      # @param {type:"raw"}
dstfr_list.loc[4] = [0,0]      # @param {type:"raw"}
dstfr_list.loc[5] = [0,0]      # @param {type:"raw"}

# Enter deformation due to settlement parameters
deform_list.loc[1] = [0,0,0]
deform_list.loc[2] = [0,0,0]
deform_list.loc[3] = [0,0,0]
deform_list.loc[4] = [0,0,0]
deform_list.loc[5] = [0,0,0]
deform_list.loc[6] = [0,0,0]
deform = np.array(deform_list).flatten()

# Enter düzgün sıcaklık değişmesi parameters
tmprt_prpts = {'t': 20, 'eps': 10**-5}
print(tmprt_prpts['t'])



p0_global = np.zeros((n_nd*3,1))

q_global = np.array(pntfr_list).reshape(n_nd*3,1)

#for ele in np.arange(1,n_el+1,1):
for ele in np.arange(1,n_el+1,1):
  i = ele_list.loc[ele]['node_i']
  j = ele_list.loc[ele]['node_j']
  EI = ele_list.loc[ele]['EI']
  EF = ele_list.loc[ele]['EF']
  beta = ele_list.loc[ele]['beta']
  L = ele_list.loc[ele]['Length']
  eps = tmprt_prpts['eps']
  dzgn_tmp = tmprt_prpts['t']
  L_x = L*ssp.cosdg(beta)
  L_z = L*ssp.sindg(beta)

  # distributed load
  W_x = dstfr_list.loc[ele]['W_x']
  W_z = dstfr_list.loc[ele]['W_z']

  ## point load
  Pl_M_i,Pl_M_j = pntfr_list.loc[i]['P_theta'],pntfr_list.loc[j]['P_theta']
  Pl_x_i,Pl_x_j = pntfr_list.loc[i]['P_x'],pntfr_list.loc[j]['P_x']
  Pl_z_i,Pl_z_j = pntfr_list.loc[i]['P_z'],pntfr_list.loc[j]['P_z']


  ## deformation due to settlement
  dfrm_theta_i,dfrm_x_i,dfrm_z_i = deform_list.loc[i]['Δ_rot'],deform_list.loc[i]['Δ_x'],deform_list.loc[i]['Δ_z']
  dfrm_theta_j,dfrm_x_j,dfrm_z_j = deform_list.loc[j]['Δ_rot'],deform_list.loc[j]['Δ_x'],deform_list.loc[j]['Δ_z']


  K_ii = np.matrix(([[4*EI/L,0,6*EI/L**2],
                       [0,EF/L,0],
                       [6*EI/L**2,0,12*EI/L**3]]))

  K_jj = np.matrix(([[4*EI/L,0,6*EI/L**2],
                       [0,EF/L,0],
                       [6*EI/L**2,0,12*EI/L**3]]))
  print('K_{}{}_ii ='.format(i,j))
  display(sym.Matrix(K_ii))

  K_ij = np.matrix(([[2*EI/L,0,6*EI/L**2],
                        [0,EF/L,0],
                        [6*EI/L**2,0,12*EI/L**3]]))

  print('K_{}{}_ij ='.format(i,j))
  display(sym.Matrix(K_ij))


  K1_ii = np.float64(K_ii+np.diag(spring_list.loc[i]))
  K1_jj = np.float64(K_jj+np.diag(spring_list.loc[j]))
  K1_ij = np.float64(K_ij)
  K1_ji = K1_ij

  K0 = np.vstack((np.hstack((K_ii,K_ij)),np.hstack((K_ij,K_jj))))
  K1 = np.vstack((np.hstack((K1_ii,K1_ij)),np.hstack((K1_ji,K1_jj))))
  K_lokal = K0-np.dot(np.dot(K0,np.linalg.inv(K1)),K0)
  print('K_{}{}_LOKAL ='.format(i,j))
  display(sym.Matrix(K_lokal))
  K_ii, K_ij, K_ji, K_jj =  K_lokal[0:3,0:3],K_lokal[0:3,3:6],K_lokal[3:6,0:3],K_lokal[3:6,3:6]

  #K_1 = np.diag(np.full((6, 6), 10**50))
  #K_1[np.]

  T2_i = np.matrix(([[1,0,0],
                       [0,-ssp.cosdg(beta),ssp.sindg(beta)],
                       [0,-ssp.sindg(beta),-ssp.cosdg(beta)]]))

  T2_j = np.matrix(([[1,0,0],
                       [0,ssp.cosdg(beta),-ssp.sindg(beta)],
                       [0,ssp.sindg(beta),ssp.cosdg(beta)]]))

  print('T_2_{} ='.format(ele))
  T_2 =  np.vstack((np.hstack((T2_i,np.zeros((3,3)))),np.hstack((np.zeros((3,3)),T2_j))))
  display(sym.Matrix(T_2))

  # Calculating FEM from distributed load
  p0_dst = np.transpose(np.matrix([-W_x*L_z**2/12+W_z*L_x**2/12,
                               W_x*L_x/2+W_z*L_z/2,
                               W_x*L_z/2+W_z*L_x/2,
                               W_x*L_x**2/12-W_z*L_x**2/12+Pl_M_j,
                              -W_x*L_x/2-W_z*L_z/2,
                              -W_x*L_z/2-W_z*L_x/2]))
  p0_indrgnms = p0_dst-np.dot(np.dot(K0,np.linalg.inv(K1)),p0_dst)

  # calculating P from deformation
  w_dfrm = np.transpose(np.hstack((deform_list.loc[i],deform_list.loc[j])))
  print('w_dfrm_{} ='.format(ele))
  print(w_dfrm)

  # calculating P due to temperature change
  p0_tmprt = np.dot(T_2,np.transpose(np.array([0,-eps*dzgn_tmp*EF,0,0,-eps*dzgn_tmp*EF,0])))

  p0_dfrm = np.dot(np.dot(np.dot(T_2,K_lokal),np.transpose(T_2)),np.transpose(w_dfrm))
  p0 = p0_indrgnms+p0_dfrm+p0_tmprt
  p0_transf = p0

  p0_global[(i-1)*3] =     p0_transf[0,0]+p0_global[(i-1)*3]
  p0_global[((i-1)*3)+1] = p0_transf[0,1]+p0_global[((i-1)*3)+1]
  p0_global[((i-1)*3)+2] = p0_transf[0,2]+p0_global[((i-1)*3)+2]
  p0_global[(j-1)*3] =     p0_transf[0,3]+p0_global[(j-1)*3]
  p0_global[((j-1)*3)+1] = p0_transf[0,4]+p0_global[((j-1)*3)+1]
  p0_global[((j-1)*3)+2] = p0_transf[0,5]+p0_global[((j-1)*3)+2]

  print('―' * 50)


  K_global[3*(i-1):3*(i-1)+3,3*(i-1):3*(i-1)+3] = np.dot(np.dot(T2_i,K_ii),np.transpose(T2_i))+K_global[3*(i-1):3*(i-1)+3,3*(i-1):3*(i-1)+3]
  K_global[3*(i-1):3*(i-1)+3,3*(j-1):3*(j-1)+3] = np.dot(np.dot(T2_i,K_ij),np.transpose(T2_j))+K_global[3*(i-1):3*(i-1)+3,3*(j-1):3*(j-1)+3]
  K_global[3*(j-1):3*(j-1)+3,3*(i-1):3*(i-1)+3] = np.dot(np.dot(T2_j,K_ji),np.transpose(T2_i))+K_global[3*(j-1):3*(j-1)+3,3*(i-1):3*(i-1)+3]
  K_global[3*(j-1):3*(j-1)+3,3*(j-1):3*(j-1)+3] = np.dot(np.dot(T2_j,K_jj),np.transpose(T2_j))+K_global[3*(j-1):3*(j-1)+3,3*(j-1):3*(j-1)+3]

  display(sym.Matrix(K_global))
  display(sym.Matrix(p0_global))


  ### force vector
  #p0[i-1:i+2] = [-dstfr_list.loc[ele]['W_x']+dstfr_list.loc[ele]['W_z']+pntfr_list.loc[ele]['P_theta']]


## trimming the equation

trim_index = np.flatnonzero(rstrd)
print(trim_index)

K_global_trimmed = np.delete(np.delete(K_global,trim_index,0),trim_index,1)
print('K_global_trimmed')
print(ntl.to_ltx(K_global_trimmed))

print('p0_global_trimmed')
p0_global_trimmed = np.delete(p0_global,trim_index,0)

q_global_trimmed = np.delete(q_global,trim_index,0)

print(ntl.to_ltx(p0_global_trimmed))
print('q_global_trimmed')
print(ntl.to_ltx(q_global_trimmed))


print('d_global_trimmed')
d = np.linalg.solve(K_global_trimmed,q_global_trimmed-p0_global_trimmed)
print(ntl.to_ltx(d))
print(d)



20
K_12_ii =


Matrix([
[66666.6666666667,                0, 16666.6666666667],
[               0, 1666666.66666667,                0],
[16666.6666666667,                0, 5555.55555555556]])

K_12_ij =


Matrix([
[33333.3333333333,                0, 16666.6666666667],
[               0, 1666666.66666667,                0],
[16666.6666666667,                0, 5555.55555555556]])

K_12_LOKAL =


...

Matrix([
[1.0,    0,    0,   0,   0,    0],
[  0, -0.6,  0.8,   0,   0,    0],
[  0, -0.8, -0.6,   0,   0,    0],
[  0,    0,    0, 1.0,   0,    0],
[  0,    0,    0,   0, 0.6, -0.8],
[  0,    0,    0,   0, 0.8,  0.6]])

w_dfrm_5 =
[0 0 0 0 0 0]
――――――――――――――――――――――――――――――――――――――――――――――――――


Matrix([
[  32051.282051282,            6250.0, -3846.15384615385,  7692.30769230769,                 0,  3846.15384615385,        0,          0,        0, 8333.33333333333,    -6250.0,          0,        0,        0,          0,       0,         0,         0],
[           6250.0,  1673697.91666667,                 0,                 0, -1666666.66666667,                 0,        0,          0,        0,          21875.0,   -7031.25,          0,        0,        0,          0,       0,         0,         0],
[-3846.15384615385,                 0,  2502350.42735043, -10256.4102564103,                 0, -2350.42735042735,        0,          0,        0,                0,          0, -2500000.0,        0,        0,          0,       0,         0,         0],
[ 7692.30769230769,                 0, -10256.4102564103,  93846.1538461538,            4800.0,  6656.41025641026,        0,          0,        0,                0,          0,          0,        0,        0,          0, 20000.0,   

Matrix([
[      0],
[ 2000.0],
[ 2000.0],
[      0],
[ -800.0],
[ 1600.0],
[      0],
[ 2000.0],
[      0],
[      0],
[-2000.0],
[      0],
[      0],
[      0],
[-2000.0],
[      0],
[-1200.0],
[-1600.0]])

[ 8 12 13 14 16 17]
K_global_trimmed
\left(
\begin{array}{}
  32051.2821 &  6250.0000 & -3846.1538 &  7692.3077 &  0.0000 &  3846.1538 &  0.0000 &  0.0000 &  8333.3333 & -6250.0000 &  0.0000 &  0.0000\\
  6250.0000 &  1673697.9167 &  0.0000 &  0.0000 & -1666666.6667 &  0.0000 &  0.0000 &  0.0000 &  21875.0000 & -7031.2500 &  0.0000 &  0.0000\\
 -3846.1538 &  0.0000 &  2502350.4274 & -10256.4103 &  0.0000 & -2350.4274 &  0.0000 &  0.0000 &  0.0000 &  0.0000 & -2500000.0000 &  0.0000\\
  7692.3077 &  0.0000 & -10256.4103 &  93846.1538 &  4800.0000 &  6656.4103 &  0.0000 &  0.0000 &  0.0000 &  0.0000 &  0.0000 &  20000.0000\\
  0.0000 & -1666666.6667 &  0.0000 &  4800.0000 &  2027434.6667 &  479424.0000 &  0.0000 &  0.0000 &  0.0000 &  0.0000 &  0.0000 &  4800.0000\\
  3846.1538 &  0.0000 & -2350.4274 &  6656.4103 &  479424.0000 &  642782.4274 &  0.0000 &  0.0000 &  0.0000 &  0.0000 &  0.0000 & -3600.0000\\
  0.0000 &  0.0000 &  0.0000 &  0.0000 &  0.0000 &  0.0000 &  100000.0000 &  0.000

In [ ]:
pip install pysimplegui

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.3/993.3 kB 5.0 MB/s eta 0:00:00


In [ ]:
import PySimpleGUI as sg
import matplotlib
matplotlib.use('Agg')

sg.Window(title="Hello World", layout=[[]], margins=(100, 50)).read()

TclError: no display name and no $DISPLAY environment variable

In [ ]:
%%shell
jupyter nbconvert --to html "Copy_of_Matris_Software.ipynb"

[NbConvertApp] Converting notebook Copy_of_Matris_Software.ipynb to html
[NbConvertApp] Writing 818497 bytes to Copy_of_Matris_Software.html


In [2]:
pip install numpyarray_to_latex

  Preparing metadata (setup.py) ... done
  Created wheel for numpyarray_to_latex: filename=numpyarray_to_latex-0.1.0-py3-none-any.whl size=9701 sha256=f3796eb17a3d30779816e7686532aa8f1808c9bc0021519715f13193d091bbd8
  Stored in directory: /root/.cache/pip/wheels/5a/c3/21/fb09eb03b910fdcc908dcd977229618319c2e11e35c1d3bf0b
Successfully built numpyarray_to_latex


In [5]:
import pandas as pd
import numpy as np
import scipy.special as ssp
import sympy as sym
import numpyarray_to_latex as ntl


# Enter no. of element
n_el = 5

# Enter no. of nodes
n_nd = 6



K_global = np.zeros((n_nd*3,n_nd*3)) # Global stiffness matrix is created

nodes_list = pd.DataFrame(data=None, columns=['x_coord','z_coord'], index=np.arange(1,n_nd+1,1))
rstrd_list = pd.DataFrame(data=None, columns=['Rstrd_theta','Rstrd_x','Rstrd_z'], index=np.arange(1,n_nd+1,1))
spring_list = pd.DataFrame(data=None, columns=['Yay_theta','Yay_x','Yay_z'], index=np.arange(1,n_el+1,1))
pntfr_list = pd.DataFrame(data=np.zeros((n_nd,3),dtype='int'), columns=['P_theta','P_x','P_z'], index=np.arange(1,n_nd+1,1))
dstfr_list = pd.DataFrame(data=np.zeros((n_el,2),dtype='int'), columns=['W_x','W_z'], index=np.arange(1,n_el+1,1))
ele_list = pd.DataFrame(data=None, columns=['node_i','node_j','Length','beta','EI','EF'], index=np.arange(1,n_el+1,1))
tmprt_list = pd.DataFrame(data=None, columns=['element','t_up','t_bot','duzgun_t','EI','EF'], index=np.arange(1,n_el+1,1))
deform_list = pd.DataFrame(data=np.zeros((n_nd,3),dtype='int'), columns=['Δ_rot','Δ_x','Δ_z'], index=np.arange(1,n_nd+1,1))


# Enter Stiffness properties
EI_global = 100000
EF_global = 10000000

# Enter nodes coordinates
nodes_list.loc[1] = [4,8]
nodes_list.loc[2] = [10,8]
nodes_list.loc[3] = [0,4]
nodes_list.loc[4] = [4,4]
nodes_list.loc[5] = [4,0]
nodes_list.loc[6] = [16,0]

# Enter elements   i-j-L-β-   EI-       EF
ele_list.loc[1] = [1,2,6,0,EI_global,EF_global]
ele_list.loc[2] = [3,4,4,0,EI_global,EF_global]
ele_list.loc[3] = [5,4,4,270,EI_global,EF_global]
ele_list.loc[4] = [4,1,4,270,EI_global,EF_global]
ele_list.loc[5] = [2,6,10,np.degrees(np.arctan(8/6)),EI_global,EF_global]


# Enter Applied point loads
pntfr_list.loc[1] = [0,0,0]
pntfr_list.loc[2] = [0,0,0]
pntfr_list.loc[3] = [0,0,0]
pntfr_list.loc[4] = [0,0,0]
pntfr_list.loc[5] = [0,0,0]
pntfr_list.loc[6] = [0,0,0]


# Enter boundary conditions (restrained)
rstrd_list.loc[1] = [0,0,0]
rstrd_list.loc[2] = [0,0,0]
rstrd_list.loc[3] = [0,0,1]
rstrd_list.loc[4] = [0,0,0]
rstrd_list.loc[5] = [1,1,1]
rstrd_list.loc[6] = [0,1,1]

rstrd = np.array(rstrd_list).flatten() # try ravel also


# Enter elastic hinge locations (springs)
spring_list.loc[1] = np.array([0.2*EI_global,10**50,10**50])
spring_list.loc[2] = np.array([10**50,10**50,10**50])
spring_list.loc[3] = np.array([10**50,10**50,10**50])
spring_list.loc[4] = np.array([10**50,10**50,10**50])
spring_list.loc[5] = np.array([10**50,10**50,10**50])
spring_list.loc[6] = np.array([10**50,10**50,10**50])


# Enter Applied distributed loads
dstfr_list.loc[1] = [0,0]
dstfr_list.loc[2] = [0,0]
dstfr_list.loc[3] = [0,0]
dstfr_list.loc[4] = [0,0]
dstfr_list.loc[5] = [0,0]

# Enter sıcaklık değişmesi parameters
#tmprt_list.loc[1] =
deform_list.loc[1] = [0,0,0]
deform_list.loc[2] = [0,0,0]
deform_list.loc[3] = [0,0,0]
deform_list.loc[4] = [0,0,0]
deform_list.loc[5] = [0,-0.003,0.004]
deform_list.loc[6] = [0,0,0]
deform = np.array(deform_list).flatten()

p0_global = np.zeros((n_nd*3,1))

q_global = np.array(pntfr_list).reshape(n_nd*3,1)

#for ele in np.arange(1,n_el+1,1):
for ele in np.arange(1,n_el+1,1):
  i = ele_list.loc[ele]['node_i']
  j = ele_list.loc[ele]['node_j']
  EI = ele_list.loc[ele]['EI']
  EF = ele_list.loc[ele]['EF']
  beta = ele_list.loc[ele]['beta']
  L = ele_list.loc[ele]['Length']
  L_x = L*ssp.cosdg(beta)
  L_z = L*ssp.sindg(beta)

  # distributed load
  W_x = dstfr_list.loc[ele]['W_x']
  W_z = dstfr_list.loc[ele]['W_z']

  ## point load
  Pl_M_i,Pl_M_j = pntfr_list.loc[i]['P_theta'],pntfr_list.loc[j]['P_theta']
  Pl_x_i,Pl_x_j = pntfr_list.loc[i]['P_x'],pntfr_list.loc[j]['P_x']
  Pl_z_i,Pl_z_j = pntfr_list.loc[i]['P_z'],pntfr_list.loc[j]['P_z']


  ## deformation due to settlement
  dfrm_theta_i,dfrm_x_i,dfrm_z_i = deform_list.loc[i]['Δ_rot'],deform_list.loc[i]['Δ_x'],deform_list.loc[i]['Δ_z']
  dfrm_theta_j,dfrm_x_j,dfrm_z_j = deform_list.loc[j]['Δ_rot'],deform_list.loc[j]['Δ_x'],deform_list.loc[j]['Δ_z']


  K_ii = np.matrix(([[4*EI/L,0,6*EI/L**2],
                       [0,EF/L,0],
                       [6*EI/L**2,0,12*EI/L**3]]))

  K_jj = np.matrix(([[4*EI/L,0,6*EI/L**2],
                       [0,EF/L,0],
                       [6*EI/L**2,0,12*EI/L**3]]))
  print('K_{}{}_ii ='.format(i,j))
  print(ntl.to_ltx(K_ii))

  K_ij = np.matrix(([[2*EI/L,0,6*EI/L**2],
                        [0,EF/L,0],
                        [6*EI/L**2,0,12*EI/L**3]]))

  print('K_{}{}_ij ='.format(i,j))
  print(ntl.to_ltx(K_ij))


  K1_ii = np.float64(K_ii+np.diag(spring_list.loc[i]))
  K1_jj = np.float64(K_jj+np.diag(spring_list.loc[j]))
  K1_ij = np.float64(K_ij)
  K1_ji = K1_ij

  K0 = np.vstack((np.hstack((K_ii,K_ij)),np.hstack((K_ij,K_jj))))
  K1 = np.vstack((np.hstack((K1_ii,K1_ij)),np.hstack((K1_ji,K1_jj))))
  K_lokal = K0-np.dot(np.dot(K0,np.linalg.inv(K1)),K0)
  print('K_{}{}_LOKAL ='.format(i,j))
  print(ntl.to_ltx(K_lokal))
  K_ii, K_ij, K_ji, K_jj =  K_lokal[0:3,0:3],K_lokal[0:3,3:6],K_lokal[3:6,0:3],K_lokal[3:6,3:6]

  #K_1 = np.diag(np.full((6, 6), 10**50))
  #K_1[np.]

  T2_i = np.matrix(([[1,0,0],
                       [0,-ssp.cosdg(beta),ssp.sindg(beta)],
                       [0,-ssp.sindg(beta),-ssp.cosdg(beta)]]))

  T2_j = np.matrix(([[1,0,0],
                       [0,ssp.cosdg(beta),-ssp.sindg(beta)],
                       [0,ssp.sindg(beta),ssp.cosdg(beta)]]))

  print('T_2_{} ='.format(ele))
  T_2 =  np.vstack((np.hstack((T2_i,np.zeros((3,3)))),np.hstack((np.zeros((3,3)),T2_j))))
  print(ntl.to_ltx(T_2))

  # Calculating FEM from distributed load
  p0_dst = np.transpose(np.matrix([-W_x*L_z**2/12+W_z*L_x**2/12,
                               W_x*L_x/2+W_z*L_z/2,
                               W_x*L_z/2+W_z*L_x/2,
                               W_x*L_x**2/12-W_z*L_x**2/12+Pl_M_j,
                              -W_x*L_x/2-W_z*L_z/2,
                              -W_x*L_z/2-W_z*L_x/2]))
  p0_indrgnms = p0_dst-np.dot(np.dot(K0,np.linalg.inv(K1)),p0_dst)

  # calculating P from deformation
  w_dfrm = np.transpose(np.hstack((deform_list.loc[i],deform_list.loc[j])))
  print('w_dfrm_{} ='.format(ele))
  print(ntl.to_ltx(w_dfrm))

  p0_dfrm = np.dot(np.dot(np.dot(T_2,K_lokal),np.transpose(T_2)),np.transpose(w_dfrm))
  p0 = p0_indrgnms+p0_dfrm
  p0_transf = p0

  p0_global[(i-1)*3] =     p0_transf[0,0]+p0_global[(i-1)*3]
  p0_global[((i-1)*3)+1] = p0_transf[0,1]+p0_global[((i-1)*3)+1]
  p0_global[((i-1)*3)+2] = p0_transf[0,2]+p0_global[((i-1)*3)+2]
  p0_global[(j-1)*3] =     p0_transf[0,3]+p0_global[(j-1)*3]
  p0_global[((j-1)*3)+1] = p0_transf[0,4]+p0_global[((j-1)*3)+1]
  p0_global[((j-1)*3)+2] = p0_transf[0,5]+p0_global[((j-1)*3)+2]

  print('―' * 50)


  K_global[3*(i-1):3*(i-1)+3,3*(i-1):3*(i-1)+3] = np.dot(np.dot(T2_i,K_ii),np.transpose(T2_i))+K_global[3*(i-1):3*(i-1)+3,3*(i-1):3*(i-1)+3]
  K_global[3*(i-1):3*(i-1)+3,3*(j-1):3*(j-1)+3] = np.dot(np.dot(T2_i,K_ij),np.transpose(T2_j))+K_global[3*(i-1):3*(i-1)+3,3*(j-1):3*(j-1)+3]
  K_global[3*(j-1):3*(j-1)+3,3*(i-1):3*(i-1)+3] = np.dot(np.dot(T2_j,K_ji),np.transpose(T2_i))+K_global[3*(j-1):3*(j-1)+3,3*(i-1):3*(i-1)+3]
  K_global[3*(j-1):3*(j-1)+3,3*(j-1):3*(j-1)+3] = np.dot(np.dot(T2_j,K_jj),np.transpose(T2_j))+K_global[3*(j-1):3*(j-1)+3,3*(j-1):3*(j-1)+3]

  print(ntl.to_ltx(K_global))
  print(ntl.to_ltx(p0_global))


  ### force vector
  #p0[i-1:i+2] = [-dstfr_list.loc[ele]['W_x']+dstfr_list.loc[ele]['W_z']+pntfr_list.loc[ele]['P_theta']]


## trimming the equation
trim_index = np.flatnonzero(rstrd)
print("trim_index")
print(trim_index)

K_global_trimmed = np.delete(np.delete(K_global,trim_index,0),trim_index,1)
print('K_global_trimmed')
print(ntl.to_ltx(K_global_trimmed))

print('p0_global_trimmed')
p0_global_trimmed = np.delete(p0_global,trim_index,0)

q_global_trimmed = np.delete(q_global,trim_index,0)

print(ntl.to_ltx(p0_global_trimmed))
print('q_global_trimmed')
print(ntl.to_ltx(q_global_trimmed))


print('d_global_trimmed')
d = np.linalg.solve(K_global_trimmed,q_global_trimmed-p0_global_trimmed)
print(ntl.to_ltx(d))
print(d)



K_12_ii =
\left(
\begin{array}{}
  66666.6667 &  0.0000 &  16666.6667\\
  0.0000 &  1666666.6667 &  0.0000\\
  16666.6667 &  0.0000 &  5555.5556
\end{array}
\right)
K_12_ij =
\left(
\begin{array}{}
  33333.3333 &  0.0000 &  16666.6667\\
  0.0000 &  1666666.6667 &  0.0000\\
  16666.6667 &  0.0000 &  5555.5556
\end{array}
\right)
K_12_LOKAL =
\left(
\begin{array}{}
  15384.6154 &  0.0000 &  3846.1538 &  7692.3077 &  0.0000 &  3846.1538\\
  0.0000 &  1666666.6667 &  0.0000 &  0.0000 &  1666666.6667 &  0.0000\\
  3846.1538 &  0.0000 &  2350.4274 &  10256.4103 &  0.0000 &  2350.4274\\
  7692.3077 &  0.0000 &  10256.4103 &  53846.1538 &  0.0000 &  10256.4103\\
  0.0000 &  1666666.6667 &  0.0000 &  0.0000 &  1666666.6667 &  0.0000\\
  3846.1538 &  0.0000 &  2350.4274 &  10256.4103 &  0.0000 &  2350.4274
\end{array}
\right)
T_2_1 =
\left(
\begin{array}{}
  1.0000 &  0.0000 &  0.0000 &  0.0000 &  0.0000 &  0.0000\\
  0.0000 & -1.0000 &  0.0000 &  0.0000 &  0.0000 &  0.0000\\
  0.0000 &  -0.0000

In [11]:
x= pd.DataFrame([[0,0,0,0,0,0],
                 [0,0,0,0,0,0],
                 [0,0,0,0,0,0]],
                columns=['Msol','Nsol','Vsol','Msağ','Nsağ','Vsağ'],
                index=['Ç12','Ç23','Ç41']
                )
#
print(x.to_latex())

\begin{tabular}{lrrrrrr}
\toprule
{} &  Msol &  Nsol &  Vsol &  Msağ &  Nsağ &  Vsağ \\
\midrule
Ç12 &     0 &     0 &     0 &     0 &     0 &     0 \\
Ç23 &     0 &     0 &     0 &     0 &     0 &     0 \\
Ç41 &     0 &     0 &     0 &     0 &     0 &     0 \\
\bottomrule
\end{tabular}



<ipython-input-11-dc77323062ee>:8: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(x.to_latex())


In [ ]:
## çubuk 12, i=1,j=2

p12_1x = np.dot(K12_1x1x,d_1x)+np.dot(K12_1x2x,d_2x)+p0_1x
p12_2x = np.dot(K12_2x2x,d_2x)+np.dot(K12_2x1x,d_1x)+p0_2x

p12_1=np.dot(np.transpose(T2_12_1),np.transpose(p12_1x))
p12_2=np.dot(np.transpose(T2_12_2),np.transpose(p12_2x))

## çubuk 23, i=2,j=3

p23_2x = np.dot(K23_2x2x,d_2x)+np.dot(K23_2x3x,d_3x)+p0_2x
p23_3x = np.dot(K23_3x3x,d_3x)+np.dot(K23_3x2x,d_2x)+p0_3x

p23_2=np.dot(np.transpose(T2_23_2),np.transpose(p23_2x))
p23_3=np.dot(np.transpose(T2_23_3),np.transpose(p23_3x))

## çubuk 41, i=4,j=1

p41_4x = np.dot(K41_4x4x,d_4x)+np.dot(K41_4x1x,d_1x)+p0_4x
p41_1x = np.dot(K41_1x1x,d_1x)+np.dot(K41_1x4x,d_4x)+p0_1x

p41_4=np.dot(np.transpose(T2_41_4),np.transpose(p41_4x))
p41_1=np.dot(np.transpose(T2_41_1),np.transpose(p41_1x))



#p_1,p_2,p_3,p_4,p_5,p_3=p41_1+p12_1,p23_2,p34_3,p41_4+p54_4,p54_5,p23_3